In [ ]:
from os import listdir
from os.path import isfile, join
#Para ver las palabras
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
# nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords #Listas de stopwords
from nltk.tokenize import word_tokenize #Tokens
import re #regex

In [ ]:
def get_txt(path):
    """
    Regresa una lista con el contenido de todos los archivos de un directorio

    Args:
        path (str): ruta de la carpeta
    """
    text = []
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    
    for file in onlyfiles:
         with open(path+"/"+file, 'rb') as f:
            text.append(f.read().decode('utf-8', 'replace'))
    return text

# Guardamos cada pelicula en un diccionario
# cada entrada del diccionario es una lista con las peliculas leidas
movies = {}
movies["Pride & Prejudice"] = get_txt("../corpus/Pride & Prejudice")
movies["Marvel"] = get_txt("../corpus/Marvel")
movies["Christopher Nolan"] = get_txt("../corpus/Christopher Nolan")

In [ ]:
def show_words(word):
    '''
    Usa la librería wordcloud para mostrar una gráfica
    de las palabras más usadasen un corpus
    '''
    mycloud = WordCloud()
    mycloud.generate_from_frequencies(Counter(word))

    plt.figure(figsize=(8,6), dpi=120)
    plt.imshow(mycloud)
    plt.axis("off")
    plt.show()

In [ ]:
def get_tokens(name):
    '''
    Separa los textos de cada película en una entrada
    del diccionario movies, usando la función word_tokenize
    '''
    corpus = []
    for i, movie in enumerate(movies[name]):
        movies[name][i] = word_tokenize(movie)
        corpus += movies[name][i]
    return corpus

# Mezclamos los corpus de cada película según su autor
marvel = get_tokens("Marvel")
nolan = get_tokens("Christopher Nolan")
pride = get_tokens("Pride & Prejudice")

In [ ]:
#Gráfica de palabras para la peli (texto sucio)
show_words(pride) #dividir por palabras
show_words(marvel)
show_words(nolan)

In [ ]:
num = len(pride) + len(marvel) + len(nolan)
print("Número de tokens antes de limpiar el texto: ", num)
num = len(set(pride + marvel + nolan))
print("Número de tokens no repetidos (tipos): ", num)

In [ ]:
# tokens que necesitan ser limpiados del corpus y que no
# se encuentran en la lista de stopwords
more = ["'ve", "'ll", "'t", "'s", "'re", "'", "'m", "'d", "n't", "oh", "hey", "yeah","okay", "mr.", "miss", "mrs."]
stopwords_list = stopwords.words('english') + more

def clean_corpus(corpus):
    '''
    Limpia el corpus recibido de stopwords
    '''
    clean = []
    pattern = r'[^a-z0-9\s]'
    for w in corpus:
        #quita stopwords y convierte a minúsculas
        if w.lower() not in stopwords_list and re.sub(pattern,'', w) != '':
            if  w == "na": #Para juntar gon na, wan na, etc.
                clean[-1] += w
            else:
                clean.append(w)
    return clean

# Limpiamos los corpus de cada película
movies["Pride & Prejudice"] = clean_corpus(movies["Pride & Prejudice"][0])
movies["Marvel"] = [clean_corpus(m) for m in movies["Marvel"]]
movies["Christopher Nolan"] = [clean_corpus(m) for m in movies["Christopher Nolan"]]

In [ ]:
def merge_movies(name):
    '''
    Mezcla el corpus de cada película en una entrada del
    diccionario movies en un solo corpus
    '''
    corpus = []
    for movie in movies[name]:
        corpus += movie
    return corpus

# Mostramos la gráfica de palabras de los corpus limpios
show_words(movies["Pride & Prejudice"])
show_words(merge_movies("Marvel"))
show_words(merge_movies("Christopher Nolan"))

# Algoritmo BPE

In [ ]:
def get_dic(corpus):
    '''
    Obtiene el diccionario del corpus dado
    en formato palabra -> frecuencia
    '''
    # agregamos espacios a cada token
    corpus = [' '.join(token) for token in corpus]

    # Creamos el diccionario
    return Counter(corpus)

dic_prejudice = get_dic(movies['Pride & Prejudice'])
dic_marvel = get_dic(merge_movies('Marvel'))
dic_nolan = get_dic(merge_movies('Christopher Nolan'))

In [ ]:
def get_pairs(sigma):
    '''
    Obtiene los pares y su frecuencia dentro del diccionario
    '''
    pairs = {}
    for word, freq in sigma.items():
        chars = word.split()
        for i in range(len(chars)-1):
            # par!
            aux = (chars[i], chars[1+i])
            pairs[aux] = freq if aux not in pairs else pairs[aux] + freq

    return pairs


def merge_frequent(pair, dic):
    '''
    Sustituye el par recibido dentro del diccionario
    '''

    new_dic = {}
    for word in dic:
        #(a, b) -> 'ab'
        new_word = word.replace(' '.join(pair), ''.join(pair))
        # define un nuevo diccionario con el nuevo simbolo ab
        new_dic[new_word] = dic[word]
    
    return new_dic
        

def BPE(epoch, dic):
    '''
    Algoritmo BPE

    Args:
        epoch (int): número de iteraciones que va a correr el algoritmo
        dic (Counter): el diccionario parabra -> frecuencia
    '''
    new_dic = dic
    for _ in range(epoch):
        # Obtenemos los pares y sus frecuencias
        pairs = get_pairs(new_dic)
        try:
            #Símbolo de mayor frecuencia
            best = max(pairs, key=pairs.get)
            # sustituimos 'a b' por 'ab' en el dicionario
            new_dic = merge_frequent(best, new_dic)
        except:
            break
    return new_dic

In [ ]:
# Pruebas
print(BPE(20, dic_prejudice))
#print(BPE(10, dic_prejudice))
#print(BPE(10, dic_prejudice))

In [ ]:
dic_nolan